In [1]:
# Import dependencies
from pathlib import Path
import pandas as pd
import numpy as np

# Import preprocessing techniques
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

# Import all metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Data Visualization
import pydotplus
from IPython.display import Image

In [3]:
# Read file into dataframe
loan_df = pd.read_csv(Path('loans_data_encoded.csv'))

In [4]:
# Display dataframe
loan_df.head()

,amount,term,age,bad,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
0,1000,30,45,0,6,0,1,0,0,0,1
1,1000,30,50,0,7,1,0,0,0,1,0
2,1000,30,33,0,8,1,0,0,0,1,0
3,1000,15,27,0,9,0,0,0,1,0,1
4,1000,30,28,0,10,0,0,0,1,1,0


In [5]:
# Check for null data
loan_df.isnull().mean()*100

amount                            0.0
term                              0.0
age                               0.0
bad                               0.0
month_num                         0.0
education_Bachelor                0.0
education_High School or Below    0.0
education_Master or Above         0.0
education_college                 0.0
gender_female                     0.0
gender_male                       0.0
dtype: float64

In [6]:
# Let's analyze the bad columns by looking at the distribution of outcomes as well as means across features
loan_df['bad'].value_counts()

0    300
1    200
Name: bad, dtype: int64

In [7]:
# The distribution of the targets look relatively close.

In [8]:
# Means across features in terms of targets
loan_df.groupby('bad').mean()

,amount,term,age,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
bad,,,,,,,,,,
0,935.0,22.116667,31.210,6.480,0.136667,0.41,0.010,0.443333,0.176667,0.823333
1,955.5,23.885000,30.975,6.695,0.130000,0.43,0.005,0.435000,0.120000,0.880000


In [9]:
# We can see that the mean of education master or above, gender_female look diverse enough between the two outcome. This is nothing conclusive but can point towards a well
#-defined prediction model. 

In [10]:
# Standardize features
scaler = StandardScaler()

In [12]:
# Define features and target in terms of X and Y
X = loan_df.drop(columns='bad', axis=1)
Y = loan_df['bad']

In [13]:
# Fit X into scaler
scaler.fit(X)

# Transform X
X = scaler.transform(X)

In [19]:
# Define training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, stratify=Y, test_size=0.2, random_state=2)

In [32]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    
    # Fit the model and print the accuracy score for each learning rate with respect to training and testing data
    classifier.fit(X_train, Y_train.ravel())
    
    training_prediction = classifier.predict(X_train)
    training_accuracy_score = accuracy_score(training_prediction, Y_train)
    testing_prediction = classifier.predict(X_test)
    testing_accuracy_score = accuracy_score(testing_prediction, Y_test)
    
    # Print each result based the learning rate
    print(f'The learning rate is: {learning_rate}.')
    print(f'The training accuracy score is {training_accuracy_score}.')
    print(f'The testing accuracy score is {testing_accuracy_score}.\n')

The learning rate is: 0.05.
The training accuracy score is 0.6075.
The testing accuracy score is 0.6.

The learning rate is: 0.1.
The training accuracy score is 0.6225.
The testing accuracy score is 0.59.

The learning rate is: 0.25.
The training accuracy score is 0.7075.
The testing accuracy score is 0.58.

The learning rate is: 0.5.
The training accuracy score is 0.74.
The testing accuracy score is 0.55.

The learning rate is: 0.75.
The training accuracy score is 0.775.
The testing accuracy score is 0.56.

The learning rate is: 1.
The training accuracy score is 0.775.
The testing accuracy score is 0.56.



In [35]:
# We will be using the learning rate of 0.75 assuming it yields the best testing result

In [37]:
# Print result into a dataframe
loan_prediction = pd.DataFrame({'Prediction':testing_prediction,
                               'Actual':Y_test
                              }).head(20)

In [38]:
loan_prediction

,Prediction,Actual
393,1,1
238,1,0
190,0,0
33,0,0
350,1,1
218,0,0
419,1,1
50,1,0
185,0,0
447,1,1


In [40]:
# Use the confusion matrix
print(confusion_matrix(testing_prediction, Y_test))

[[40 24]
 [20 16]]


In [41]:
# Use the classification report
print(classification_report(testing_prediction, Y_test))

              precision    recall  f1-score   support

           0       0.67      0.62      0.65        64
           1       0.40      0.44      0.42        36

    accuracy                           0.56       100
   macro avg       0.53      0.53      0.53       100
weighted avg       0.57      0.56      0.56       100

